In [1]:
# adjust the R limit
memory.limit(size= 81011630)

[1] 81011630

In [2]:
#necessary library
library(data.table) #fread
library(lubridate)
library(reshape2)
library(dplyr)
library(scales)
library(stringr)
#library(tidyverse)

Warning message:
"package 'data.table' was built under R version 3.6.3"

Attaching package: 'lubridate'


The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following object is masked from 'package:base':

    date


Warning message:
"package 'reshape2' was built under R version 3.6.3"

Attaching package: 'reshape2'


The following objects are masked from 'package:data.table':

    dcast, melt



Attaching package: 'dplyr'


The following objects are masked from 'package:lubridate':

    intersect, setdiff, union


The following objects are masked from 'package:data.table':

    between, first, last


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




## Bin speeds and aggregate table

In [2]:
interval_df = read.csv("../../data/tidy/train-trajectory-19.csv")
dc = fread("../../data/raw/energy-consumption-08-19.csv") # Read in energy data

ERROR: Error: cannot allocate vector of size 1000.0 Mb


In [22]:
# select random days
set.seed(1234)
sub = sample(1:nrow(interval_df),720)
interval_df_test = interval_df[sub,]

In [9]:
interval_df$speed_mph = interval_df$kph*0.621371 #kph to mph
interval_df$distance_mile = interval_df$dist*0.000621371 #convert from meters to mile
interval_df$time_hr = interval_df$timeinterval/3600.0 #convert from seconds to hour

In [10]:
# Calculate the speed bins 
bin_speeds <- function (dataframe) {
    dummy_cols = c(paste0("speed_bin_",1:5,"_dummy"))
    bin_time_cols = c(paste0("speed_bin_",1:5,"_time_hr"))
    testcuts <- quantile(dataframe$speed_mph,seq(0,1,.2),na.rm=TRUE) 
    print(paste0("The speed bins are: ", testcuts))
    dataframe[, 'speed_bin_1_dummy' := sapply(speed_mph,  function(x) ifelse (x < testcuts[2], 1, 0))]
    dataframe[, 'speed_bin_2_dummy' := sapply(speed_mph,  function(x) ifelse (x >= testcuts[2] & x < testcuts[3], 1, 0))]
    dataframe[, 'speed_bin_3_dummy' := sapply(speed_mph,  function(x) ifelse (x >= testcuts[3] & x < testcuts[4], 1, 0))]
    dataframe[, 'speed_bin_4_dummy' := sapply(speed_mph,  function(x) ifelse (x >= testcuts[4] & x < testcuts[5], 1, 0))]
    dataframe[, 'speed_bin_5_dummy' := sapply(speed_mph,  function(x) ifelse (x >= testcuts[5], 1, 0))]
    dataframe[, (bin_time_cols) := lapply(.SD, function(x) x * dataframe$time_hr ), .SDcols = dummy_cols]
    print(paste0("Percentage error of summed speed bin times = ", round(100*(sum(colSums(dataframe[, bin_time_cols],na.rm=TRUE)) 
               - sum(dataframe$time_hr, na.rm=TRUE))/sum(dataframe$time_hr, na.rm=TRUE),2), "%"))                                                  
    return(dataframe)
}

In [11]:
# Calculate the acceleration bins
bin_accelerations <- function (dataframe) {
    dummy_cols = c(paste0("accel_bin_",1:5,"_dummy"))
    bin_time_cols = c(paste0("speed_bin_",1:5,"_time_hr"))
    testcuts <- quantile(dataframe$acc,seq(0,1,.2),na.rm=TRUE) 
    print(paste0("The acceleration bins are: ", testcuts))
    dataframe[, 'accel_bin_1_dummy' := sapply(acc,  function(x) ifelse (x < testcuts[2], 1, 0))]
    dataframe[, 'accel_bin_2_dummy' := sapply(acc,  function(x) ifelse (x >= testcuts[2] & x < testcuts[3], 1, 0))]
    dataframe[, 'accel_bin_3_dummy' := sapply(acc,  function(x) ifelse (x >= testcuts[3] & x < testcuts[4], 1, 0))]
    dataframe[, 'accel_bin_4_dummy' := sapply(acc,  function(x) ifelse (x >= testcuts[4] & x < testcuts[5], 1, 0))]
    dataframe[, 'accel_bin_5_dummy' := sapply(acc,  function(x) ifelse (x >= testcuts[5], 1, 0))]
    dataframe[, (bin_time_cols) := lapply(.SD, function(x) x * dataframe$time_hr ), .SDcols = dummy_cols]
    print(paste0("Percentage error of summed acceleration bin times = ", round(100*(sum(colSums(dataframe[, bin_time_cols ],na.rm=TRUE)) 
               - sum(dataframe$time_hr, na.rm=TRUE))/sum(dataframe$time_hr, na.rm=TRUE),2), "%"))                                                  
    return(dataframe)
}

In [12]:
# Aggregate dataframe at hour level
hour_aggregate <- function (dataframe) {
    sum_cols = c("distance_mile","time_hr",paste0("speed_bin_",1:5,"_time_hr"),paste0("accel_bin_",1:5,"_time_hr"))
    agg_df = dataframe[, lapply( .SD, sum , na.rm=TRUE), by = c('Month','Hour','Day'), .SDcols = sum_cols]
    avg_interval_speed_mph_df = dataframe[, lapply( .SD, mean , na.rm=TRUE), by = c('Month','Hour','Day'), .SDcols = 'speed_mph']
    agg_df[, 'avg_interval_speed_mph'] = avg_interval_speed_mph_df$speed_mph
    agg_df[, 'avg_hour_speed_mph'] = agg_df$distance_mile/agg_df$time_hr
    return(agg_df)
}

In [13]:
# Combine energy consumption data (MAKE THIS A FUNCTION)
merge_energy <- function (hour_dt, energy_df) {    
    # Melt by hour 
    melted_energy_df = melt(energy_df, id.vars=c('Year','Month', 'Day of Month','WJ','TAVG'),measure.vars = paste0("Hour ", 1:24))
    colnames(melted_energy_df) = c('Year', 'Month', 'Day', 'WJ', 'TAVG', 'Hour', 'energy_kwh')
    hour_energy_dt <- setDT(melted_energy_df)
    hour_energy_dt[, Hour := str_replace(Hour, "Hour ", "")]
    hour_energy_dt$Hour = as.numeric(hour_energy_dt$Hour)
    hour_energy_dt$Hour = hour_energy_dt$Hour - 1
    merged_dt = merge(hour_dt, hour_energy_dt, all=T)
    return(merged_dt)
}

## Computing the final table

In [14]:
#df_test = subset(interval_df,interval_df$Month==4)
energy2019 = subset(dc,dc$Year==2019)
## WRITE df_final as a CSV.
#write.csv(df_test_final, 'df_test_final_hour_agg_April.csv')

In [15]:
main <- function (movements_df, energy_df) {
    final_df <- movements_df %>% bin_speeds() %>% bin_accelerations() %>% hour_aggregate() %>% merge_energy(energy_df)
    return(final_df)
}

In [ ]:
df_final = main(interval_df,energy2019)
write.csv(df_final,"../Data/tidy/system-model.csv")